In [37]:
# 원티드 크롤링
import requests
import datetime

def crawling_wanted_data(parsed_data, info_keys):
    result={}
    for key in info_keys:
        if key == 'company':   # 회사 이름
            result['company'] = parsed_data['job']['company']['name']  
        elif key == 'intro':   # 회사 소개
            result['intro'] = parsed_data['job']['detail']['intro'] 
        elif key == 'location':  # 근무 지역
            result['location'] = parsed_data['job']['address']['full_location'] 
        elif key == 'title':  # 제목
            result['title'] = parsed_data['job']['position']
        elif key == 'main_job': # 주요업무
            result['main_job'] = parsed_data['job']['detail']['main_tasks']
        elif key == 'require':  # 자격요건
            result['require'] = parsed_data['job']['detail']['requirements']
        elif key == 'prefer_point':  # 우대사항
            result['prefer_point'] = parsed_data['job']['detail']['preferred_points']
        elif key == 'benefits':  # 혜택 및 복지
            result['benefits'] = parsed_data['job']['detail']['benefits']
        elif key == 'deadline':  # 마감날짜 None = 상시
            result['deadline'] = parsed_data['job']['due_time']
        elif key == 'geo_location': # 근무지역 (위도,경도)
            geo_location = parsed_data['job']['address'].get('geo_location')
            if geo_location:
                result['lat'] = geo_location['n_location']['lat']
                result['lng'] = geo_location['n_location']['lng']
            else:
                result['lat'] = None
                result['lng'] = None
    return result

parsed_data = []

timestamp = int(datetime.datetime.now().timestamp())
tag_type_ids = 518
years = -1
limit = 100
offset = 0
# tag_type_ids: 518 개발/ 507 경영-비즈니스 /523 마케팅 /511 디자인 /530 영업 /510 고객서비스-리테일 /524 미디어 /513 엔지니어링-설계
# 517 HR /959 게임 제작 /508 금융 /522 제조-생산 /515 의료-제약-바이오 /10101 교육 /532 물류-무역 /10057 식-음료 /521 법률-법집행기관 /509 건설-시설 /514 공공-복지
# limit -> 100이면 100개씩 데이터추출 (한번에 100밖에 불가)
# offset -> 0번째부터 추출

# 전체 데이터 수를 예측하거나 특정 수로 제한합니다. 예를 들어, 총 300개의 데이터를 가져오고 싶다면:
total_data_count = 10000

for offset in range(0, total_data_count, limit):
    
    url = "https://www.wanted.co.kr/api/v4/jobs?{}&country=kr&tag_type_ids={}&job_sort=job.latest_order&locations=all&years={}&limit={}&offset={}".format(timestamp, tag_type_ids, years, limit, offset)
    parse_company = requests.get(url).json()
    
    recur_ids = [temp['id'] for temp in parse_company['data']]
    

    for recur_id in recur_ids:
        detail_url = f"{base_url}{recur_id}?{timestamp}"
        job_data = requests.get(detail_url).json()

        info_keys = ['company', 'intro']  # 필요한 정보 키
        crawled_data = crawling_wanted_data(job_data, info_keys)

        print(crawled_data)


{'company': '백퍼센트', 'intro': '백퍼센트는 2019년 8월 설립해 2020년 2월  보조배터리 공유 서비스 `충전돼지`를 론칭하였습니다.  이후 극장(CGV, 메가박스, 롯데시네마), 편의점(이마트 24, 미니스톱, 세븐일레븐), 백화점(현대백화점, 롯데백화점),  놀이공원(롯데월드, 서울랜드) 등 다양한 보조배터리 공유 공간을 창출하며 새로운 사업모델을 성공적으로 선도하고 있습니다. \n\n충전돼지는 공유 보조배터리 서비스 업계 1위로써, 누적 80억원 이상의 투자를 달성하며 성장 가능성을 시장에서 인정받고 있으며 서비스 론칭 이후 누적 이용회원 200만 달성 누적 400만 시간의 배터리 공유(대여) 등 매월 20%씩 누적 성장을 기록하고 있습니다.\n\n충전돼지는 공간을 스마트하고 가치있게 만들 수 있는 참신한 아이디어와 열정을 갖춘 모두에게 열려있는 기업입니다.\n\n많은 관심과 지원 부탁드립니다.'}
{'company': '카카오헬스케어', 'intro': '카카오헬스케어는 IT를 통해 세상을 더 건강하게 만들기 위하여 설립되었습니다.\n\n우리는 디지털 헬스케어 산업을 혁신하기 위해서는 개개인과 병원, 연구기관 및 제약회사 등 모든 Player들을 바라보아야 한다고 생각합니다.\n\n이러한 이유로 우리는 아래의 3가지 서비스를 만들고 있습니다.\n\n#라이프 로그 데이터를 통해 개인의 긍정적인 변화를 만들 수 있는 Virtual Care 서비스\n#병원의 디지털 의료 환경과 환자 경험 개선을 위한 Digital Transformation 서비스\n#의료 데이터를 기반으로 더 정확하고 빠른 의료 연구를 지원하는 Data Enabler 서비스\n\n창립 2년 차의 역사가 짧은 카카오헬스케어가 다수의 글로벌 파트너들과 함께 일하고 있다는 사실만으로도 우리가 올바른 방향으로 계속해서 나아가고 있고, 결실을 맺을 능력이 있음을 입증하고 있습니다. \n \n현재 우리의 비전을 실현시키기 위하여 의료계, 학계, 공공기관, 대기업 및 글로벌 기